# Strava Exports to CSV Files
Now that all of our data is uncompressed, we have to transform all of the seperate files to something that can be loaded into a database. MySQL's `LOAD DATA INFILE` is quite efficient for bulk loading, so if we are able to transform the data for each table into a CSV file then loading the data should be more straightforward.

As we get into going through data for various users it gets more difficult to keep track of what data is supposed to represent and who it belongs to, so I think an object oriented approach might be better. In the end, the strat may be to load all the data for each table into a pandas DataFrame, use the `pandas.to_csv()` method, and then import the generated CSVs into MySQL.

For information and advice on the three filetypes used and their contents, this tutorial was heavily referenced: [Parsing fitness tracker data with Python](https://towardsdatascience.com/parsing-fitness-tracker-data-with-python-a59e7dc17418/)

In [21]:
'''
Package Imports, Constants, Global Variable

Run this cell to import all the packages we need and define some constants. 
You'll likely need to install any missing packages to your Python environment
with pip or your package manager of choice.
'''

import os
import gpxpy
import fitdecode
import pandas as pd
import csv
from collections import namedtuple
import random

DATA_DIR_PATH = os.path.join('..', 'data')  # Path of data directory relative to this Jupyter Notebook
ACTIVITY_DIR_PATH = os.path.join(DATA_DIR_PATH, 'export_activities') # Parent directory of all exports
OUTPUT_DIR_PATH = os.path.join(DATA_DIR_PATH, 'outputs')
cur_activity_id = 0   # Global activity counter to give each activity a unique id across users
MYSQL_NULL = 'NULL'

# Namedtuple to pass around fields we want to end up in the activities table
Activity_Fields = namedtuple('Activity_Fields', ['user_id', 
                                                 'activity_id', 
                                                 'name', 
                                                 'type', 
                                                 'description', 
                                                 'filename'])

Let's define an `Activity` object. We will use this object to store all the data from an individual file from an export, whether it is a .fit, .gpx, or .tcx file. By feeding the path to the activity file in the constructor, we are able to make an `Activity` create itself from a file when it is instantiated.

In [22]:
class Activity:

  __mysql_null = 'NULL'
  __activity_summary_keys = ['start_datetime', 'end_datetime', 
                             'distance_2d', 'distance_3d',
                             'avg_speed', 'max_speed',
                             'uphill', 'downhill',
                             'avg_hr', 'min_hr', 'max_hr',
                             'avg_cad','min_cad','max_cad',
                             'total_kcal']


  def __init__(self, activity_fields: Activity_Fields, activity_filepath: os.PathLike):
    self.__activity_id = activity_fields.activity_id
    self.__activity_filepath = activity_filepath
    self.__points_df = pd.DataFrame()

    self.__activity_summary = activity_fields._asdict()
    
    global MYSQL_NULL
    self.__activity_summary.update(dict.fromkeys(self.__activity_summary_keys,
                                                 MYSQL_NULL))

    self.__point_dict = {
      'activity_id': [],
      'latitude': [],
      'longitude': [],
      'elevation': [],
      'time': [],
      'speed': [],
      'hr': [],
      'cad': []
    }

    self.__load_from_file()
    

  def __load_from_file(self) -> None:
    match self.__activity_filepath.split('.')[-1].lower():
      case 'gpx':
        self.__load_from_gpx()
      case 'tcx':
        self.__load_from_tcx()
      case 'fit':
        self.__load_from_fit()


  def __load_from_gpx(self) -> None:

    with open(self.__activity_filepath) as f:
      gpx = gpxpy.parse(f)

      uphill, downhill = 0, 0

      if len(gpx.tracks) == 0:
        raise ValueError(f'No tracks found in gpx file {os.path.abspath(self.__activity_filepath)}')

      for track in gpx.tracks:

        uphill_downhill = track.get_uphill_downhill()
        uphill += uphill_downhill.uphill
        downhill += uphill_downhill.downhill
        
        for segment in track.segments:
          for point_idx, point in enumerate(segment.points):            
            self.__point_dict['activity_id'].append(self.__activity_id)
            self.__point_dict['time'].append(point.time)
            self.__point_dict['latitude'].append(point.latitude)
            self.__point_dict['longitude'].append(point.longitude)
            self.__point_dict['elevation'].append(point.elevation)

            # Adding speed
            point_speed = point.speed
            if point_idx == 0:
              point_speed = 0
            elif point_speed == None:
              point_speed = point.speed_between(segment.points[point_idx - 1])
            self.__point_dict['speed'].append(round(point_speed, 3))

            # Adding extensions
            found_hr = False
            found_cad = False
            for extension in point.extensions:

              hr_element = extension.find('{http://www.garmin.com/xmlschemas/TrackPointExtension/v1}hr')
              cad_element = extension.find('{http://www.garmin.com/xmlschemas/TrackPointExtension/v1}cad')
              
              # Adding heart rate, if exists
              if hr_element != None and hr_element.text:
                self.__point_dict['hr'].append(int(hr_element.text))
                found_hr = True
              
              # Adding cadence, if exists
              if cad_element != None and cad_element.text:
                self.__point_dict['cad'].append(int(cad_element.text))
                found_cad = True
            
            # Adding nulls if cadence or heart rate don't exist
            if not found_hr:
              self.__point_dict['hr'].append('\\N')
            if not found_cad:
              self.__point_dict['cad'].append('\\N')
      
      # Creating the dataframe of points from the dictionary
      self.__points_df = pd.DataFrame(self.__point_dict)

      # Populating activity summary dictionary
      timebounds = gpx.get_time_bounds()
      self.__activity_summary.update({'start_datetime': timebounds.start_time,
                                      'end_datetime': timebounds.end_time,
                                      'distance_2d': round(gpx.length_2d(), 3),
                                      'distance_3d': round(gpx.length_3d(), 3),
                                      'avg_speed': round(self.__points_df['speed'].mean(), 3),
                                      'max_speed': round(self.__points_df['speed'].max(), 3),
                                      'uphill': round(uphill, 3),
                                      'downhill': round(downhill, 3)})
      
      questionable_cols = ['hr', 'cad']
      for col in questionable_cols:
        if(self.__points_df[col].dtype == 'int64'):
          self.__activity_summary.update({'avg_' + col: round(self.__points_df[col].mean(), 3),
                                          'min_' + col: self.__points_df[col].min(),
                                          'max_' + col: self.__points_df[col].max()})
    

    # Ensure activity_id is stored as integer
    self.__points_df['activity_id'] = self.__points_df['activity_id'].astype(int)


  # Stubs to be replaced with real file parsing code
  def __load_from_tcx(self) -> None:
    pass


  def __load_from_fit(self) -> None:
    pass


  # Getter methods
  def get_summary(self) -> dict:
    return self.__activity_summary
  

  def get_points(self) -> pd.DataFrame:
    return self.__points_df

Now let's define a `User`. An entire export directory of activity files belongs to a Strava user, so our `User` can have a list of `Activities`. By feeding the path to the export directory into the constructor, a `User` is able to initialize itself with all of its `Activities` upon instantiation.

In [23]:
class User:

  __user_field_keys = ['user_id', 
                       'email_address', 
                       'first_name', 
                       'last_name', 
                       'description', 
                       'weight', 
                       'city', 
                       'state', 
                       'country']


  def __init__(self, export_filepath: os.PathLike):
    self.__export_filepath = export_filepath

    self.__user_id = 0
    self.__user_fields = {}
    self.__load_user_info( anonymized=True)

    self.__activities = []
    self.__load_all_activities()

    self.__load_challenges()

    self.__load_follows()


  def __load_user_info(self, anonymized = False):
    profile_csv = csv.DictReader(open(os.path.join(self.__export_filepath, 'profile.csv'), encoding = 'utf-8'))
    for row in profile_csv:    
      if anonymized:
        self.__user_fields = dict.fromkeys(self.__user_field_keys, 'sample_data (anonymized)')
      else:
        global MYSQL_NULL
        self.__user_fields = dict.fromkeys(self.__user_field_keys, MYSQL_NULL)
        for key in self.__user_field_keys[1:]:
          colname = ' '.join(key.split('_')).title()
          self.__user_fields[key] = row[colname] if row[colname] != '' else MYSQL_NULL

      self.__user_id = self.__user_fields['user_id'] = row['Athlete ID']
      break


  def __load_all_activities(self):
    # Read in activities.csv for this User
    activity_csv_df = pd.read_csv(os.path.join(self.__export_filepath, 'activities.csv'),
                                  usecols=['Activity Name', 
                                           'Activity Type',
                                           'Activity Description',
                                           'Filename'])
    files = os.listdir(self.__export_filepath)
    for file in files:
      # Only process known file types
      file_ext = file.split('.')[-1].lower()
      if file_ext in ['gpx']:#, 'tcx', 'fit']:
        try:
          global cur_activity_id
          df_row = activity_csv_df[activity_csv_df['Filename'].str.contains(file, na=False)]
          cur_activity_fields = Activity_Fields(user_id=self.__user_id, 
                                                activity_id=cur_activity_id,
                                                name=df_row['Activity Name'].iloc[0],
                                                type=df_row['Activity Type'].iloc[0],
                                                description=df_row['Activity Description'].iloc[0],
                                                filename=file)
          self.__activities.append(Activity(activity_fields=cur_activity_fields,
                                            activity_filepath = os.path.join(self.__export_filepath, file)))
          cur_activity_id += 1
        except ValueError as ve:
          print(f'Error occured when loading {file}: {ve}')


  def __load_challenges(self):
    colnames = ['join_datetime', 'name', 'completed']
    global_challenges_csv_df = pd.read_csv(os.path.join(self.__export_filepath, 'global_challenges.csv'),
                                           header = 0,
                                           names = colnames,
                                           parse_dates = ['join_datetime'])
    group_challenges_csv_df = pd.read_csv(os.path.join(self.__export_filepath, 'group_challenges.csv'),
                                          header = 0,
                                          names = colnames,
                                          parse_dates = ['join_datetime'])
    global_challenges_csv_df['type'] = 'global'
    group_challenges_csv_df['type'] = 'group'
    challenge_dfs = [global_challenges_csv_df, group_challenges_csv_df]
    nonempty_challenge_dfs = [df for df in challenge_dfs if not df.empty]
    self.__combined_challenge_df = pd.concat(nonempty_challenge_dfs)
    self.__combined_challenge_df['user_id'] = self.__user_id
    self.__combined_challenge_df['completed'] = self.__combined_challenge_df['completed'].astype(int)

  def __load_follows(self):
    colnames = ['follow_status', 'favorite_status']
    following_csv_df = pd.read_csv(os.path.join(self.__export_filepath, 'following.csv'),
                                   header = 0,
                                   names = ['followee_user_id'] + colnames,
                                   usecols = ['followee_user_id', 'follow_status'])
    following_csv_df['follower_user_id'] = self.__user_id
    followers_csv_df = pd.read_csv(os.path.join(self.__export_filepath, 'followers.csv'),
                                   header = 0,
                                   names = ['follower_user_id'] + colnames,
                                   usecols = ['follower_user_id', 'follow_status'])
    followers_csv_df['followee_user_id'] = self.__user_id
    self.__follow_df = pd.concat([following_csv_df, followers_csv_df])


  # Getters for exporting
  def get_follows(self) -> pd.DataFrame:
    return self.__follow_df

  def get_challenges(self) -> pd.DataFrame:
    return self.__combined_challenge_df


  def get_activity_summaries(self) -> pd.DataFrame:
    activity_summaries = [activity.get_summary() for activity in self.__activities]
    return pd.DataFrame(activity_summaries)


  def get_activity_points(self) -> pd.DataFrame:
    point_dfs = [activity.get_points() for activity in self.__activities if not activity.get_points().empty]
    return pd.concat(point_dfs)
  
  
  def get_user_fields(self) -> dict:
    return self.__user_fields

Keeps track of multiple `Users`

In [ ]:
class Secretary:

  def __init__(self, all_exports: os.PathLike):
    self.__all_exports_path = all_exports
    self.__users = []
    self.__load_all_users()

  def __load_all_users(self):
    dirs = os.listdir(self.__all_exports_path)
    i = 1
    for dir in dirs:
      print(f'Loading user {i} of {len(dirs)} ...')
      self.__users.append(User(os.path.join(self.__all_exports_path, dir)))
      i += 1
    print('Done loading in all users!')

  def export_all_to_csvs(self, dest_path: os.PathLike):
    self.__create_points_csv(os.path.join(dest_path, 'points.csv'))
    self.__create_activities_csv(os.path.join(dest_path, 'activities.csv'))
    self.__create_users_csv(os.path.join(dest_path, 'users.csv'))
    self.__create_challenges_csv(os.path.join(dest_path, 'challenges.csv'))
    self.__create_follows_csv(os.path.join(dest_path, 'follows.csv'))

  def __create_points_csv(self, dest_path: os.PathLike):
    point_dfs = [user.get_activity_points() for user in self.__users if not user.get_activity_points().empty]
    combined_points_df = pd.concat(point_dfs)
    combined_points_df.to_csv(dest_path, index_label='seq_num', lineterminator='\n')

  def __create_activities_csv(self, dest_path: os.PathLike):
    activity_dfs = [user.get_activity_summaries() for user in self.__users if not user.get_activity_summaries().empty]
    combined_activities_df = pd.concat(activity_dfs)
    combined_activities_df.to_csv(dest_path, index = False, lineterminator='\n')

  def __create_users_csv(self, dest_path: os.PathLike):
    all_user_fields = [user.get_user_fields() for user in self.__users]
    combined_users_df = pd.DataFrame(all_user_fields)
    combined_users_df.to_csv(dest_path, index = False)

  def __create_challenges_csv(self, dest_path: os.PathLike):
    challenge_dfs = [user.get_challenges() for user in self.__users if not user.get_challenges().empty]
    combined_challenges_df = pd.concat(challenge_dfs, ignore_index=True)
    combined_challenges_df.to_csv(dest_path, index_label='challenge_id')

  def __create_follows_csv(self, dest_path: os.PathLike):
    follow_dfs = [user.get_follows() for user in self.__users]
    combined_follows_df = pd.concat(follow_dfs)
    user_ids = [int(user.get_user_fields()['user_id']) for user in self.__users]
    print(user_ids)
    print(combined_follows_df)
    self.__combined_follows_df = combined_follows_df
    self.__combined_follows_df = combined_follows_df[(combined_follows_df['follower_user_id'].isin(user_ids)) & 
                                                     (combined_follows_df['followee_user_id'].isin(user_ids))]
    print(self.__combined_follows_df)
    self.__combined_follows_df.to_csv(dest_path, index = False)

  def get_follows(self) -> pd.DataFrame:
    return self.__combined_follows_df

Let's test it out by initializing a `User`:

In [25]:
# steve = User(export_filepath = '../data/export_activities/export_101635319')

In [26]:
# print(steve.get_follows().info())
# follows_df = steve.get_follows()
# follows_df.to_csv('../data/follows.csv', index = False)

And let's look at the activities summaries CSV for this user:

In [27]:
# points_df = steve.get_activity_points()
# points_df.to_csv('../data/points.csv', index_label='seq_num', lineterminator='\n')

# activities_df = steve.get_activity_summaries()
# print(activities_df.head(15))
# activities_df.to_csv('../data/activities_summaries.csv', index = False, lineterminator='\n')

In [28]:
secretary = Secretary(all_exports = os.path.abspath(ACTIVITY_DIR_PATH))

Loading user 1 of 3 ...
Error occured when loading 7077892227.gpx: No tracks found in gpx file c:\Users\matth\Documents\Python\CS3200\CS3200_Strava_Secretary\data\export_activities\export_101635319\7077892227.gpx


C:\Users\matth\AppData\Local\Temp\ipykernel_21492\1958986006.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  global_challenges_csv_df = pd.read_csv(os.path.join(self.__export_filepath, 'global_challenges.csv'),


Loading user 2 of 3 ...
Loading user 3 of 3 ...


C:\Users\matth\AppData\Local\Temp\ipykernel_21492\1958986006.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  global_challenges_csv_df = pd.read_csv(os.path.join(self.__export_filepath, 'global_challenges.csv'),


Done loading in all users!


In [29]:
# print(os.path.exists('../data/export_activities'))
# print(ACTIVITY_DIR_PATH)
os.makedirs(OUTPUT_DIR_PATH, exist_ok=True)
secretary.export_all_to_csvs(os.path.join('..', 'data', 'outputs'))

C:\Users\matth\AppData\Local\Temp\ipykernel_21492\1958986006.py:121: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(point_dfs)


['101635319', '57141745', '96589216']
    followee_user_id follow_status follower_user_id
0           36064991      Accepted        101635319
1           41868570      Accepted        101635319
2           45050667      Accepted        101635319
3           45711813      Accepted        101635319
4           47388563      Accepted        101635319
..               ...           ...              ...
130         96589216      Accepted        148511532
131         96589216      Accepted        150679523
132         96589216      Accepted        153835805
133         96589216      Accepted        158514221
134         96589216      Accepted        159639266

[903 rows x 3 columns]
    followee_user_id follow_status follower_user_id
0           36064991      Accepted        101635319
1           41868570      Accepted        101635319
2           45050667      Accepted        101635319
3           45711813      Accepted        101635319
4           47388563      Accepted        101635319
..

In [49]:
follows = secretary.get_follows()

user_ids = ['101635319', '57141745', '96589216']
user_ids = [int(id) for id in user_ids]
print(user_ids)
combined_follows_df = follows[(follows['followee_user_id'].isin(user_ids))]
print(combined_follows_df)
combined_follows_df = combined_follows_df[(follows['follower_user_id'].isin(user_ids))]
print(combined_follows_df)

[101635319, 57141745, 96589216]
    followee_user_id follow_status follower_user_id
10          57141745      Accepted        101635319
23          96589216      Accepted        101635319
206         96589216      Accepted         57141745
212        101635319      Accepted         57141745
42          57141745      Accepted         96589216
73         101635319      Accepted         96589216
Empty DataFrame
Columns: [followee_user_id, follow_status, follower_user_id]
Index: []


C:\Users\matth\AppData\Local\Temp\ipykernel_21492\4122973669.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  combined_follows_df = combined_follows_df[(follows['follower_user_id'].isin(user_ids))]
